In [1]:
import os, sys, json
autocog_home = os.getenv("AUTOCOG_HOME", '/workspace/projects/autocog')
sys.path.append(autocog_home)

from autocog import CogArch
from autocog.lm import OpenAI, Llama, TfLM
from autocog.architecture.utility import PromptTee

from autocog.utility.pynb import wrap_graphviz
from IPython.display import display

arch = CogArch(pipe=PromptTee(prefix='test', tee=sys.stdout))

In [2]:
cog = arch.load(tag='test', language='sta', program="""
entry(formulate): You are answering a question in 3 stages: `formulate`, `reflect`, and `submit`. You have the possibility to iterate between `formulate` and `reflect` until you are satisfied with your answer. You do not have access to any tool and must use your own knowledge to answer.

formats:
- sentence: One sentence per line, if too long continue on the next line. A list of sentence forms a paragraph, a blank sentence indicate the start of a new paragraph.

prompt(formulate): In the current stage, `formulate`, you write an answer to a question. You are given your previous work and your evaluation of this work.
- target(question) source(?question)
- target(evaluation) source(reflect.evaluation)
- target(evaluation.answer) source(.response.answer)
> question(text): a user's question
> evaluation[0,1](record): your evaluation of your previous version of the answer
> > answer[{Na}](sentence): your previous answer
> > critics[0,{Nc}](thought): your critics of that answer
> > why(sentence): your justification as to why that answer is ready or not
> work[1,{Nw}](record): prepare what has to be done to answer the question
> > scratch[{Ns}](thought): consider what has to be done to formulate a good answer to the question.
> > task(sentence): write down one task
> response(record): Below, you write you new answer to the question
> > answer[{Na}](sentence): Use your work to redact an answer to the user's question
__next(reflect): You will evaluate your answer and provide comments if the answer needs to be improved.

prompt(reflect): In the current stage, `reflect`, you comment your proposed answer then decide whether to `formulate` another answer or `submit` the proposed one.
- target(question) source(?question)
- target(answer) source(formulate.response.answer)
> question(text): a user's question
> answer[{Na}](sentence): your proposed answer
> evaluation[1](record): You will critic this answer then evaluate whether it is ready to submit.
> > critics[1,{Nc}](thought): evaluate how this answer satisfy the question. Include pertinent comments that will help create a better answer if needed.
> > ready(bool): do you think the answer is ready to submit?
> > why(sentence): in one sentence, should you `submit` this answer as it is or should you `formulate` a new answer?
__next(formulate[{L}],submit): should you `formulate` a new answer or `submit` your current answer

prompt(submit): 
- target(answer) source(reflect.answer)
> answer[{Na}](sentence):  
__exit(answer): 

?Na=10
?Ns=3
?Nw=3
?Nc=5
?L=20
""")
# cog.model_dump()

stag_to_ordered_children={'formulate': ['question.0', 'evaluation.1', 'work.2', 'response.3'], 'evaluation.1': ['answer.1.0', 'critics.1.1', 'why.1.2'], 'work.2': ['scratch.2.0', 'task.2.1'], 'response.3': ['answer.3.0']}
parent_stag=formulate
  child_stag=question.0
  child_vs=label='question' desc="a user's question" fmt='text' path=[(0, 'question')] is_list=False list_range=None
  child_stag=evaluation.1
  child_vs=label='evaluation' desc='your evaluation of your previous version of the answer' fmt='record' path=[(1, 'evaluation')] is_list=True list_range=(0, 1)
    src_stag=question.0
    src_vs=label='question' desc="a user's question" fmt='text' path=[(0, 'question')] is_list=False list_range=None
      tgt_stag=answer.1.0
      tgt_vs=label='answer' desc='your previous answer' fmt='sentence' path=[(1, 'evaluation'), (0, 'answer')] is_list=True list_range=(10, 10)
      > True False False False
    src_stag=why.1.2
    src_vs=label='why' desc='your justification as to why that an

In [3]:
from autocog.utility.pynb import wrap_graphviz
dotstr = arch.toGraphViz()
#print(dotstr)
wrap_graphviz(dotstr)

In [4]:
# llama_path = lambda x: "/workspace/models/llama/{}/ggml-model-{}.bin".format(*x)
# model_path = llama_path(('7B','q4_0'))
model_path = '/workspace/models/openalpaca/3b_600bt_preview/ggml-model-f32.bin'
model_path = '/workspace/models/openllama/3B/ggml-model-f16.bin'
model_path = '/workspace/models/llama-2-TheBloke/7B/llama-2-7b.ggmlv3.q8_0.bin'
model_kwargs = Llama.create(model_path=model_path, n_ctx=2048)
arch.orchestrator.LMs.update({
  'text'     : Llama(**model_kwargs, completion_kwargs={ 'max_tokens' :  0 }),
  'thought'  : Llama(**model_kwargs, completion_kwargs={ 'max_tokens' : 20, 'temperature' : 1.0 }),
  'sentence' : Llama(**model_kwargs, completion_kwargs={ 'max_tokens' : 50, 'temperature' : 0.3 }),
  'markdown' : Llama(**model_kwargs, completion_kwargs={ 'max_tokens' : 50, 'temperature' : 0.1 }),
}) # llama-cpp-python defaults: top_p=0.95, top_k=40, repeat_penalty=1.1

llama.cpp: loading model from /workspace/models/llama-2-TheBloke/7B/llama-2-7b.ggmlv3.q8_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 7 (mostly Q8_0)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_internal: mem required  = 8500.72 MB (+ 1026.00 MB per state)
llama_new_context_with_model: kv self size  = 1024.00 MB


In [7]:
# await arch('test', question='What is existentialism?')
await arch('test', question='What is the capital of France?')



 === test[1] === 

You are a helpful AI assistant. You are answering a question in 3 stages: `formulate`, `reflect`, and `submit`. You have the possibility to iterate between `formulate` and `reflect` until you are satisfied with your answer. You do not have access to any tool and must use your own knowledge to answer. In the current stage, `formulate`, you write an answer to a question. You are given your previous work and your evaluation of this work.
You are using an interactive questionnaire.
Follow this structure after the start prompt:
```
> question(text): a user's question
> evaluation[0,1](record): your evaluation of your previous version of the answer
> > answer[10](sentence): your previous answer
> > critics[0,5](thought): your critics of that answer
> > why(sentence): your justification as to why that answer is ready or not
> work[1,3](record): prepare what has to be done to answer the question
> > scratch[3](thought): consider what has to be done to formulate a good answ

AssertionError: Expect index to write a list

In [6]:
# [ ''.join(x['answer']) for x in arch.orchestrator.frames[1].stacks['formulate'] ]